# J2P Power Train

## Global Calculation Setup

In this section we are going to define some basic and globally valid constants for the J2P Bot calculations. This includes for example the dimensions of the bot, the weight and also some constant factors for the different friction scenarios.

In [121]:
import math
import numpy as np

# J2P Constants
TOTAL_MASS     = 10    # [kg]
TOTAL_LENGTH   = 0.700 # [m]
TOTAL_WIDTH    = 0.540 # [m]
TOTAL_HEIGHT   = 0.120 # [m]

WHEEL_DIAMETER = 0.12  # [m]
WHEEL_RADIUS   = WHEEL_DIAMETER / 2.0 # [m]

# General constants
GRAVITY        = 9.810 # m/s^2

# Friction
STATIC_FH_WHEEL_STREET   = 0.80 # Source: https://www.schweizer-fn.de/stoff/reibwerte/reibwerte.php
STATIC_FH_WHEEL_GRAVEL   = 0.45 # Schlupt 20% Quelle: Diplomarbeit Daniel Wallner TU Graz

SLIDING_FG_WHEEL_STREET  = 0.50 # Sliding friction on dry concrete Source: https://www.schweizer-fn.de/stoff/reibwerte/reibwerte.php

ROLLING_FRR_WHEEL_STREET = 0.015 # Rolling resistance of car tire on asphalt. Source: https://de.wikipedia.org/wiki/Rollwiderstand
ROLLING_f_WHEEL_STREET   = 0.008 # Hebelarm der Rollreibung Source: Tabellenbuch Metall mit Formelsammlung - Digitales Buch, Europa-Nr. 10609V, 48. Auflage

# Environment
DEFAULT_SLOPE = 0 # [deg] Zero degree -> no slope
ROBOT_LOAD    = 0 # [kg] Load to the robot (e.g. electronics, battery, camera etc.)

# Global Scenarios
DEFAULT_EPSILON  = 0.1 # 10% addon because of bearing and supports
DEFAULT_VELOCITY = 2.5 # m/s
DEFAULT_ACCEL    = 0.5 # m/s^2

## Weight

In this section we are going to calculate the weight, the normal weight force and also the downhill slope force.

The weight: $\vec{F}_G = m * \vec{g}$

The normal weight force: $\vec{F}_{GN} = cos(\alpha) * \vec{F}_G$

The downhill slope force $\vec{F}_{GH} = sin(\alpha) * \vec{F}_G$

In the Python code below, we have the constants <code>TOTAL_MASS + ROBOT_LOAD</code> for $m$, <code>GRAVITY</code> for $g$ and <code>DEFAULT_SLOPE</code> for the slope angle $\alpha$

In [122]:
Total_FG = (TOTAL_MASS + ROBOT_LOAD) * GRAVITY
Total_FGN = math.cos(math.radians(DEFAULT_SLOPE)) * Total_FG
Total_FGH = math.sin(math.radians(DEFAULT_SLOPE)) * Total_FG

print("FG : {:3.4} N".format(Total_FG))
print("FGN: {:3.4} N".format(Total_FGN))
print("FGH: {:3.4} N".format(Total_FGH))

FG : 98.1 N
FGN: 98.1 N
FGH: 0.0 N


Due to the fact that we assume a homogenous distribution of the weight, we can calculate the normal weight force and the downhill slope force for a single wheel simply by dividing the corresponding forces by 6

$\vec{F}_{GNWheel} = \frac{\vec{F}_{GN}}{6}$

$\vec{F}_{GHWheel} = \frac{\vec{F}_{GH}}{6}$

In [123]:
Wheel_FGN = Total_FGN / 6
Wheel_FGH = Total_FGH / 6

print("FGN: {:3.4} N".format(Wheel_FGN))
print("FGH: {:3.4} N".format(Wheel_FGH))

FGN: 16.35 N
FGH: 0.0 N


## Friction

### Static Friction

The first calculation covers the static friction. For this we use only friction factors for car tires on asphalt. As we know, this friction if limiting the maximum torque we can apply to the wheels before they slip.

$\vec{F}_{HWheel} = \mu_H * \vec{F}_{GNWheel}$

In [124]:
# Defining the MU_H values
MU_H = STATIC_FH_WHEEL_STREET # Friction factor of car tire on dry asphalt
#MU_H = 0.5 # Friction factor of car tire on wet asphalt
#MU_H = 0.1 # Friction factor of car tire on ice

Wheel_FH = MU_H * Wheel_FGN

print("Wheel_FH: {:3.4} N".format(Wheel_FH))

Wheel_FH: 13.08 N


### Sliding Friction

The calculation of the sliding friction is nearly the same as for the static friction. Only the friction factor changes. This friction occurs if the wheel of the bot start slipping. If we apply to much torque or if we break to hard.

$\vec{F}_{GRWheel} = \mu_G * \vec{F}_{GNWheel}$

In [125]:
# Defining the MU_G values
MU_G = SLIDING_FR_WHEEL_STREET # Sliding Friction factor of car tire on dry asphalt

Wheel_FGR = MU_G * Wheel_FGN

print("Wheel_FGR: {:3.4} N".format(Wheel_FGR))

Wheel_FGR: 8.175 N


### Rolling Friction
The roling friction calculation is from the mathematical point of view very similar to the other friction types. But the friction factor is completely different and is based on completely different physical concepts

$\vec{F}_{RRWheel} = \mu_R * \vec{F}_{GNWheel}$

$\mu_R$ is defined by the following relationship:

$\mu_R = \frac{f}{R}$

Hereby $f$ is called the "Lever of the rolling friction" and $R$ is the wheel radius <code>WHEEL_RADIUS</code>

In [126]:
# Defining the cR value
MU_R = ROLLING_f_WHEEL_STREET / WHEEL_RADIUS
#CR = ROLLING_FR_WHEEL_STREET # rolling friction factor of car tire on dry asphalt

Wheel_FRR = CR * Wheel_FGN

print("Wheel_FRR: {:3.4} N".format(Wheel_FRR))

Wheel_FRR: 0.218 N


## Movement Scenarios

In this section we are going to define and calculat a couple of exampe scenarios for the movement of the J2P Bot. This is necessary to identify the worst case scenario in respect to the needed torque because this has a major impact on the selection of the DC motor and gearbox.

### Scenario A: Movement on flat, horizontal surface

The first scenario is the simplest scenario we could imagine (beside the one where the robot stands still ;-) ). In this scenario the J2P Bot is moving on a flat horizontal surface without any obstacles. Hereby we start from $v = 0$ which means the robot needs to accelerate from zero to $v_{max}$

One important aspect for the scenario calculation is: Due to the fact that the J2P Bot has 6 wheels and each wheel has a motor, we are calculating the scenario only for a single wheel because that is what the DC motor has to power.

At first we calculate the force which would be needed to accelerate the J2P Bot with the specified acceleration (mapped to a single wheel!).

$\vec{F}_{A} = m * \vec{a} = \frac{F_{GNWheel}}{g} * \vec{a}$

After that, we add the resistances like rolling resistance and a possible downhill slop force.

$\vec{F}_{AReal} = \vec{F}_{A} + \vec{F}_{RRWheel} + \vec{F}_{GHWheel}$

And after that, we add a percentaged value of the just calculated total force to consider also some friction and support losses in the mechanics.

$\vec{F}_{AReal} = \vec{F}_{AReal} + \epsilon$

We also need to check whether the force which we would need to move one wheel is still smaller or equal the maximum static friction.

$F_{RWheelmax} \geq \vec{F}_{AReal}$

In [127]:
# The pure force for the theoretical movement without resistances
Wheel_FA = (Wheel_FGN / GRAVITY) * DEFAULT_ACCEL
# The force including the resistances (rolling resistance and downhill slope force)
Wheel_FA_Real = Wheel_FA + Wheel_FRR + Wheel_FGH
# Add the bearing and support losses
Wheel_FA_Real = Wheel_FA_Real + (Wheel_FA_Real * DEFAULT_EPSILON)

# Check whether the force is smaller or equal to FR
NotSlipping = Wheel_FA_Real <= Wheel_FR

# Calculate the needed torque
Wheel_MA = Wheel_FA_Real * WHEEL_RADIUS

print("Wheel_FA     : {:3.4} N".format(Wheel_FA))
print("Wheel_FA_Real: {:3.4} N".format(Wheel_FA_Real))
print("Not Slipping : {}".format(NotSlipping))

print("Wheel_MA     : {:3.4} Nm".format(Wheel_MA))

Wheel_MA_Stored = Wheel_MA

Wheel_FA     : 0.8333 N
Wheel_FA_Real: 1.156 N
Not Slipping : True
Wheel_MA     : 0.06939 Nm


### Scenario B: Movement across a small obstacle

The second scenario is very similar to the first one but this time we have to overcome a small obstacle (like a door sill). For this an additional force steps into the game. This is the downhill slope force which is needed to "climb" over the obstacle. To calculate this, we have to do some intermediate calculation because the obstacle will be modelled like a slope with a specific angle. For this intermediate calculation we will first calculate the angle $\phi$ and a special distance $y$ which are depending on the wheel geometry and the obstacle height.

$\phi = arccos\Big(\frac{R_W - h}{R_W}\Big)$

$y = sin(\phi) * R_W$

Whereby $R_W$ is our <code>WHEEL_RADIUS</code> and $h$ is the height of the obstacle

And with $\phi$ and $y$ in place, we can calculate the angle $\alpha$ which will be the slope substitute angle for the obstacle

$\alpha = arctan\Big(\frac{h}{y}\Big)$

With the slope angle, we can easily calculate the additional "obstacle downhill slope force"

$F_{GHOWheel} = \sin(\alpha) * F_{GNWheel}$

In [128]:
# First, defining the obstacle
Obstacle_Height = 0.03 # 3cm height

# Now calculating the angle phi in the wheel circle
cosPhi = (WHEEL_RADIUS - Obstacle_Height) / WHEEL_RADIUS # (r - h) / r
phi = math.degrees(math.acos((cosPhi)))
# Calculating the distance y
y = math.sin(math.radians(phi)) * WHEEL_RADIUS
# Calculating the angle alpha for the slope
alpha = math.degrees(math.atan(Obstacle_Height / y))

print("Angle Phi         : {:3.4}°".format(phi))
print("Distance y        : {:3.4}m ({:3.4}cm)".format(y, y*100))
print("Angle Alpha       : {:3.4}°".format(alpha))

# Calculate the downhill slope force for the obstacle (based on alpha)
Wheel_FGH_Obstacle = (math.sin(math.radians(alpha)) * Total_FG) / 6
print("Wheel_FGH_Obstacle: {:3.4}N\n".format(Wheel_FGH_Obstacle))

Angle Phi         : 60.0°
Distance y        : 0.05196m (5.196cm)
Angle Alpha       : 30.0°
Wheel_FGH_Obstacle: 8.175N



As soon as we have the obstacle downhill slope force, we can just add it to the other resistance forces

$\vec{F}_{AReal} = \vec{F}_{A} + \vec{F}_{RRWheel} + \vec{F}_{GHWheel} + \vec{F}_{GHOWheel} + \epsilon$

The rest of the calculation is completely the same as for the horizontal movement without an obstacle. 

Important: We reduced the acceleration in this scenario to a fourth of the original one because usually you drive more slowly across an obstacle.

In [129]:
# The pure force for the theoretical movement without resistances
Wheel_FA = (Wheel_FGN / GRAVITY) * (DEFAULT_ACCEL / 4.0) # Use only a 4th of the acceleration
# The force including the resistances (rolling resistance and downhill slope force + downhill clopse force obstacle)
Wheel_FA_Real = Wheel_FA + Wheel_FRR + Wheel_FGH + Wheel_FGH_Obstacle
# Add the bearing and support losses
Wheel_FA_Real = Wheel_FA_Real + (Wheel_FA_Real * DEFAULT_EPSILON)

# Check whether the force is smaller or equal to FR
NotSlipping = Wheel_FA <= Wheel_FR

# Calculate the needed torque
Wheel_MA = Wheel_FA_Real * WHEEL_RADIUS

print("Wheel_FA     : {:3.4} N".format(Wheel_FA))
print("Wheel_FA_Real: {:3.4} N".format(Wheel_FA_Real))
print("Not Slipping : {}".format(NotSlipping))

print("Wheel_MA     : {:3.4} Nm".format(Wheel_MA))
print("Factor of normal to obstacle: {:3.4}".format(Wheel_MA / Wheel_MA_Stored))

Wheel_FA     : 0.2083 N
Wheel_FA_Real: 9.461 N
Not Slipping : True
Wheel_MA     : 0.5677 Nm
Factor of normal to obstacle: 8.181


As you can see, the needed torque for the obstacle scenario is 8 times bigger than the one you need for the horizontal movement without an obstacle. Now you can also imagine (and also try out in the calcualtion) what would happen if you don't decrease the accerleration in this scenario.